In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as K
print(tf.__version__)
print(tf.config.list_physical_devices())
!pwd

2.4.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/content


In [ ]:
def recall_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    K = tf.keras.backend
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ln -s /content/gdrive/My\ Drive/cv_final_project /gd
!ls /gd
!mkdir -p /w
%cd /w

/
Mounted at /content/gdrive
Dataset.7z		 v_2_resnet50.ipynb	 v_4_xception_adam.ipynb
v_1_resnet50_ft40.ipynb  v_3_xception_sgd.ipynb
/w


In [ ]:
!cp /gd/Dataset.7z /w
!7z x /w/Dataset.7z -o/w/Dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /w/             1 file, 60678820 bytes (58 MiB)

Extracting archive: /w/Dataset.7z
--
Path = /w/Dataset.7z
Type = 7z
Physical Size = 60678820
Headers Size = 27545
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      5% 3 - 0/0001.jpg                    12% 185 - 0/0183.jpg                      17% 374 - 0/0372.jpg                      22% 556 - 0/0554.jpg                      25% 684 - 0/0682.jpg                      32% 737 - 0/0735.jpg                      38% 909 - 0/0907.

In [ ]:
!ls Dataset

0  1  2


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

img_height, img_width = 224, 224
batch_size = 64
nb_epochs = 60
nb_classes = 3

train_data_dir = '/w/Dataset'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.2,
    rotation_range=40,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 2183 images belonging to 3 classes.
Found 544 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50, ResNet50V2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model_fine_tune():
    baseModel = ResNet50(
        weights="imagenet", 
        include_top=False,
        input_tensor=Input(shape=(img_height, img_width, 3)),
        pooling='avg',
    )
    
    # for layer in baseModel.layers:
    #     layer.trainable = False
    # for layer in baseModel.layers[-20:]:
    #     layer.trainable = True

    headModel = baseModel.output
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(nb_classes, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)
    
    return model

loss = CategoricalCrossentropy()
optimizer = SGD(lr=1e-2, momentum=0.9, decay=1e-2/40)
model = build_model_fine_tune()
model.compile(
    loss=loss, 
    optimizer=optimizer, 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

# cb_early_stopper = EarlyStopping(monitor = 'val_f1_m', patience = 20)
cb_checkpointer = ModelCheckpoint(filepath = '/w/best_weight', monitor = 'val_f1_m', save_best_only = True, mode = 'max')

history = model.fit(
    train_generator, 
    validation_data=validation_generator, 
    epochs=nb_epochs, 
    batch_size=batch_size, 
    callbacks=[cb_checkpointer],
)

94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/60
35/35 [==============================] - 53s 1s/step - loss: 0.8057 - accuracy: 0.6816 - f1_m: 0.6354 - precision_m: 0.7140 - recall_m: 0.5898 - val_loss: 1.1672 - val_accuracy: 0.2059 - val_f1_m: 0.1994 - val_precision_m: 0.2068 - val_recall_m: 0.1927
INFO:tensorflow:Assets written to: /w/best_weight/assets
Epoch 2/60
35/35 [==============================] - 39s 1s/step - loss: 0.3320 - accuracy: 0.8727 - f1_m: 0.8662 - precision_m: 0.8993 - recall_m: 0.8363 - val_loss: 1.9920 - val_accuracy: 0.6893 - val_f1_m: 0.6962 - val_precision_m: 0.6962 - val_recall_m: 0.6962
INFO:tensorflow:Assets written to: /w/best_weight/assets
Epoch 3/60
35/35 [==============================] - 39s 1s/step - loss: 0.2137 - accuracy: 0.9163 - f1_m: 0.9125 - precision_m: 0.9206 - recall_m: 0.9046 - val_loss: 0.8879 - val_accuracy: 0.6912 - val_f1_m: 0.6940 - val_precision_m: 0.6953 - val_recall_m: 0.6927
Epoch 4/60
35/35 [===========

In [ ]:
best_model = tf.keras.models.load_model('/w/best_weight', custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
best_model.compile(
    loss=loss, 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

In [ ]:
best_model.save("/w/modelv2.h5")

In [ ]:
download("/w/modelv2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
  )

Y_pred = best_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))